# k-Means Clustering

## Objectives

- Explore the application of k-means clustering on synthetic and real-world datasets.
- Determine the optimal number of clusters using various internal validation measures.
- Demonstrate data preprocessing and dimensionality reduction techniques.

## Background

This notebook investigates the k-means clustering algorithm, an unsupervised machine learning technique for partitioning datasets into distinct clusters by minimizing intra-cluster variance.

## Datasets Used

- A synthetic dataset was generated using make_blobs with 300 samples across four clusters.
- The USArrests dataset has statistics on arrest rates for assault, murder, and rape per 100,000 residents, along with the percentage of the population living in urban areas for each of the 50 U.S. states in 1973.

## Introducing k-Means Clustering

k-means clustering is an unsupervised machine learning algorithm that partitions a dataset into k distinct, non-overlapping subgroups (clusters) by minimizing the variance within each cluster.

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import ClusterVisualizer as cv

import plotly.express as px
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

In [2]:
from sklearn.datasets import make_blobs

X, _ = make_blobs(n_samples=300, centers=4, cluster_std=0.5, random_state=0)

In [3]:
# Saving the data to a DataFrame
dataX = pd.DataFrame(X, columns=['x', 'y'])
print(dataX.shape)
dataX.head()

(300, 2)


,x,y
0,1.039925,1.929910
1,-1.386091,7.480596
2,1.125389,4.966980
3,-1.056890,7.818339
4,1.402004,1.726729


In [4]:
# Plot the data
cv1 = cv.ClusterVisualizer(dataX)

cv1.plot_data()

By eye, it is relatively easy to pick out the four clusters.

In [5]:
from sklearn.cluster import KMeans

# Let's run kMeans asking for 4 clusters
kmeans4 = KMeans(n_clusters=4, random_state=10, n_init='auto').fit(X)

In [6]:
# Get the cluster centers
centers4 = kmeans4.cluster_centers_
centers4.round(2)

array([[-1.35,  7.77],
       [-1.57,  2.85],
       [ 1.99,  0.87],
       [ 0.95,  4.4 ]])

In [7]:
# Get the cluster labels
labels4 = kmeans4.predict(X)

In [8]:
# Plot the clusters with the centers
cv1.plot_clusters(labels4, centers4, title='k-Means with 4 clusters')

As you can see, the k-means algorithm (at least in this simple case) assigns the points to clusters similarly to how we might set them by eye.

In [9]:
# Let's try with 3 clusters
kmeans3 = KMeans(n_clusters=3, random_state=10, n_init='auto').fit(X)

# Get the cluster centers
centers3 = kmeans3.cluster_centers_

# Get the cluster labels 
labels3 = kmeans3.predict(X)

# Plot the clusters with the centers
cv1.plot_clusters(labels3, centers3, title='k-Means with 3 clusters')

As you can see, two clusters merged into one. 

In [10]:
# Let's try with 2 clusters
kmeans2 = KMeans(n_clusters=2, random_state=10, n_init='auto').fit(X)

# Get the cluster centers
centers2 = kmeans2.cluster_centers_

# Get the cluster labels
labels2 = kmeans2.predict(X)

# Plot the clusters with the centers
cv1.plot_clusters(labels2, centers2, title='k-Means with 2 clusters')

k-Means algorithm gives us the number of clusters we ask for.

In [11]:
# Let's try with 5 clusters
kmeans5 = KMeans(n_clusters=5, random_state=10, n_init='auto').fit(X)

# Get the cluster centers
centers5 = kmeans5.cluster_centers_

# Get the cluster labels
labels5 = kmeans5.predict(X)

# Plot the clusters with the centers
cv1.plot_clusters(labels5, centers5, title='k-Means with 5 clusters')

In [12]:
# Let's try with 6 clusters
kmeans6 = KMeans(n_clusters=6, random_state=10, n_init='auto').fit(X)

# Get the cluster centers
centers6 = kmeans6.cluster_centers_

# Get the cluster labels
labels6 = kmeans6.predict(X)

# Plot the clusters with the centers
cv1.plot_clusters(labels6, centers6, title='k-Means with 6 clusters')

In [13]:
# Let's try with 7 clusters
kmeans7 = KMeans(n_clusters=7, random_state=10, n_init='auto').fit(X)

# Get the cluster centers
centers7 = kmeans7.cluster_centers_

# Get the cluster labels
labels7 = kmeans7.predict(X)

# Plot the clusters with the centers
cv1.plot_clusters(labels7, centers7, title='k-Means with 7 clusters')

In [14]:
# Let's try with 8 clusters
kmeans8 = KMeans(n_clusters=8, random_state=10, n_init='auto').fit(X)

# Get the cluster centers
centers8 = kmeans8.cluster_centers_

# Get the cluster labels 
labels8 = kmeans8.predict(X)

# Plot the clusters with the centers
cv1.plot_clusters(labels8, centers8, title='k-Means with 8 clusters')

The k-Means algorithm creates clusters by iteratively assigning data points to the nearest centroid of k clusters and updating the centroid positions based on the mean of the points in each cluster until convergence. How can we know the optimal number of clusters?

## Optimal Number of Clusters

One of the more challenging tasks in clustering is identifying the correct number of clusters. There are several methods for it. 

### The Elbow Plot

The elbow method is a way to estimate the k value.

Let's consider k values in a range from 1 to 20.

For each k value, we will initialize k-means and use the inertia attribute to identify the sum of squared distances of samples to the nearest cluster center.

In [15]:
inertia = [KMeans(n_clusters=k, random_state=10, n_init='auto').fit(X).inertia_ 
           for k in range(2,21)]
inertia = np.array(inertia).round(2)

In [16]:
# Elbow graph
fig_e = px.scatter(x=list(range(2, len(inertia) + 2)), y=inertia)
fig_e.update_traces(marker=dict(size=8, opacity=0.8))
fig_e.update_layout(width=600, height=400, title='Elbow Curve', 
                  xaxis_title='k', yaxis_title='Sum of Squared Distances')
fig_e.show()

The elbow is a point on the graph where the decrease in the sum of squared distances (inertia) begins to slow down, indicating that adding more clusters doesn't significantly improve the fit. 

In the previous graph, it looks like the elbow is at k=4, indicating the optimal k for this dataset is 4.

Notice that the election of the elbow can be somewhat subjective. 

In [17]:
# Find the elbow point
from kneed import KneeLocator

kn = KneeLocator(list(range(2,len(inertia)+2)), inertia, curve='convex', direction='decreasing')
print('Elbow point at:', kn.knee)

Elbow point at: 4


In [18]:
# Elbow graph with elbow point
fig_e = px.scatter(x=list(range(2, len(inertia) + 2)), y=inertia)
fig_e.update_traces(marker=dict(size=8, opacity=0.8))
fig_e.update_layout(width=600, height=400, title='Elbow Curve', 
                  xaxis_title='k', yaxis_title='Sum of Squared Distances')
fig_e.add_vline(x=kn.knee, line_width=2, line_dash="dash", line_color="grey")
fig_e.show()

## Internal Validation Meassures

Internal validation methods for clustering assess the quality of cluster assignments using metrics derived from the data without requiring external labels, focusing on cohesion and separation measures within and between clusters.

### The Silhouette Score

The Silhouette Score is a metric used to calculate the degree of separation between clusters, ranging from -1 (poor clustering) to +1 (ideal clustering), where a high score indicates that clusters are well separated and cohesive.

In [19]:
from sklearn.metrics import silhouette_score

print('The Silhouette Score for k-Means (k = 4) is %.4f' %silhouette_score(X, labels4))

The Silhouette Score for k-Means (k = 4) is 0.7357


Let's compute the Silhouette Score for several cluster configurations.

In [20]:
labels = pd.DataFrame()
labels['k=2'] = labels2
labels['k=3'] = labels3
labels['k=4'] = labels4
labels['k=5'] = labels5
labels['k=6'] = labels6
labels['k=7'] = labels7
labels['k=8'] = labels8
labels.head()

,k=2,k=3,k=4,k=5,k=6,k=7,k=8
0,1,2,2,2,2,2,2
1,0,0,0,4,4,4,5
2,1,2,3,3,5,5,7
3,0,0,0,0,0,0,0
4,1,2,2,2,2,2,2


In [21]:
cv1.plot_silhouette(labels, title='Silhouette Scores for k-Means')

According to the Silhouette score, the best number of clusters is k=4.

### The Davies-Bouldin Index

The Davies-Bouldin Score is an internal evaluation metric for clustering that quantifies the average similarity between each cluster and its most similar cluster, with lower scores indicating better clustering, and its range is from 0 to infinity, where 0 represents the ideal score.

In [22]:
from sklearn.metrics import davies_bouldin_score

print('The Davies Bouldin Score for k-Means (k = 4) is %.4f' %davies_bouldin_score(X, labels4))

The Davies Bouldin Score for k-Means (k = 4) is 0.3663


In [23]:
cv1.plot_davies_bouldin(labels, title='Davies Bouldin Scores for k-Means')

According to the Davies-Bouldin index, the best number of clusters is k=4.

### The Calinski-Harabasz Index

The Calinski-Harabasz Score, also known as the Variance Ratio Criterion, is a measure of cluster validity based on the ratio of the sum of between-cluster dispersion to within-cluster dispersion, with higher scores indicating better-defined clusters. Its range is theoretically unbounded, starting from 0 upwards.

In [24]:
from sklearn.metrics import calinski_harabasz_score

print('The Calinski-Harabasz Score for k-Means (k = 4) is %.4f' %calinski_harabasz_score(X, labels4))

The Calinski-Harabasz Score for k-Means (k = 4) is 1742.1396


In [25]:
cv1.plot_calinski_harabasz(labels, title='Calinski-Harabasz Scores for k-Means')

According to the Calinski-Harabasz index, the best number of clusters is k=4.

In this case, the results of the three internal measurements coincide.

## USArrests Dataset

The USArrests dataset contains statistics on the arrest rates for assault, murder, and rape per 100,000 residents, along with the percentage of the population living in urban areas, for each of the 50 U.S. states in 1973.

In [26]:
from pydataset import data

df = data('USArrests')
print(df.shape)
df.head()

(50, 4)


,Murder,Assault,UrbanPop,Rape
Alabama,13.2,236,58,21.2
Alaska,10.0,263,48,44.5
Arizona,8.1,294,80,31.0
Arkansas,8.8,190,50,19.5
California,9.0,276,91,40.6


In [27]:
# Converting df to a long format for visualization purposes
df_melted = df.melt(var_name='Feature', value_name='Value', ignore_index=False).reset_index()

# Plot the data
fig_o = px.box(df_melted, x='Feature', y='Value', color='Feature')
fig_o.update_layout(xaxis_title='Feature', yaxis_title='Value',
                    width=600, height=400,
                    title='USArrests Dataset - Original Data')
fig_o.show()

As you can see, the features in the datasets have different scales. We must standardize them.

In [28]:
scaler = StandardScaler()

dfS = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
dfS.head()

,Murder,Assault,UrbanPop,Rape
0,1.255179,0.790787,-0.526195,-0.003451
1,0.513019,1.118060,-1.224067,2.509424
2,0.072361,1.493817,1.009122,1.053466
3,0.234708,0.233212,-1.084492,-0.186794
4,0.281093,1.275635,1.776781,2.088814


In [29]:
# Converting df to a long format for visualization purposes
dfS_melted = dfS.melt(var_name='Feature', value_name='Value', ignore_index=False).reset_index()

# Plot the data
fig_s = px.box(dfS_melted, x='Feature', y='Value', color='Feature')
fig_s.update_layout(xaxis_title='Feature', yaxis_title='Value',
                    width=600, height=400,
                    title='USArrests Dataset - Standardized Data')
fig_s.show()

We will apply Principal Component Analysis (PCA) to reduce the dimensionality of the standardized data to two principal components, making it possible to visualize the dataset in a 2D space while preserving as much variance as possible.

In [30]:
# Applying PCA
pca = PCA(n_components=2, random_state=0)

dfS_pca = pd.DataFrame(pca.fit_transform(dfS), columns=['PCA_1', 'PCA_2'])

In [31]:
# Plotting the data in the PCA Space
cv_a = cv.ClusterVisualizer(dfS_pca)

cv_a.plot_data(title='PCA Space (2 components)')

In [32]:
# Computing the k-Means Clustering with k = 2 to 5
kM2 = KMeans(n_clusters=2, random_state=22, n_init='auto').fit(dfS)
kM3 = KMeans(n_clusters=3, random_state=22, n_init='auto').fit(dfS)
kM4 = KMeans(n_clusters=4, random_state=22, n_init='auto').fit(dfS)
kM5 = KMeans(n_clusters=5, random_state=22, n_init='auto').fit(dfS)

Notice we ran the k-Means algorithms using the standardized data to prevent larger-scale variables from dominating the clustering process.

In [33]:
# Getting the cluster labels
kM_labels = pd.DataFrame()
kM_labels['k=2'] = kM2.labels_
kM_labels['k=3'] = kM3.labels_
kM_labels['k=4'] = kM4.labels_
kM_labels['k=5'] = kM5.labels_

Let's apply metrics for each k-Means solution to evaluate the quality of clustering, with Silhouette measuring cluster cohesion and separation, Davis-Bouldin comparing within-to-between cluster distances, and Calinski-Harabasz assessing cluster density and separation.

In [34]:
cv_a.plot_silhouette(kM_labels, title='Silhouette Scores for k-Means')

In [35]:
cv_a.plot_davies_bouldin(kM_labels, title='Davies Bouldin Scores for k-Means')

In [36]:
cv_a.plot_calinski_harabasz(kM_labels, title='Calinski-Harabasz Scores for k-Means')

The three indexes choose different clustering methods. They disagree. For this exercise, we will select the clustering solution of k-Means with k=2 (two clusters).

Let's plot the two clusters identified by the k-Means algorithm (k=2) in the 2D space defined by the first two principal components from the PCA, providing a visual representation of how the data points are grouped.

In [37]:
cv_a.plot_clusters(kM2.labels_, title='k-Means with 2 clusters in the PCA Space')

Through standardization, PCA, and k-Means clustering with validation by internal measures, the analysis determined that a two-cluster solution best captures the inherent structure of the USArrests dataset in terms of arrest rates and urban population. 

Visualization in the PCA-reduced space further illustrates the distinct grouping, offering insights into the similarities and differences across the U.S. states based on the considered variables. 

This approach demonstrates the power of combining data preprocessing, dimensionality reduction, and clustering techniques to uncover meaningful patterns in multidimensional data.

But, what characterizes the two clusters? How are the original variables split into the two clusters? Let's make a plot to have an idea.

In [38]:
# Adding the k-Means labels to the standardized DataFrame
dfS['kMeans-2'] = kM2.labels_
# Transformig to a long format for visualization purposes
dfS_melted = pd.melt(dfS, id_vars=['kMeans-2'], value_vars=dfS.columns[:-1])
dfS_melted.head()

,kMeans-2,variable,value
0,1,Murder,1.255179
1,1,Murder,0.513019
2,1,Murder,0.072361
3,0,Murder,0.234708
4,1,Murder,0.281093


In [39]:
# Plot the data
fig = px.box(dfS_melted, x='kMeans-2', y='value', color='variable', 
             width=800, height=400, title='Box Plot of Standardized Data')
# Change the legend position to 'top'
fig.update_layout(legend=dict(x=0.4, y=1.2, orientation='h'))
# Show the plot
fig.show()

As you can see, cluster 0 seems to have lower values for murder, assault, and rape. 

## Conclusions

Key Takeaways:
- The k-means algorithm efficiently identifies distinct clusters, matching visual estimation.
- Optimal cluster determination via the elbow method, silhouette score, and other internal metrics suggests four clusters as the ideal number for the synthetic dataset.
- After standardization and PCA, applying k-means to the USArrests dataset revealed that a two-cluster solution effectively captures differences in arrest rates and urbanization.
- Visualizations highlight the differences between clusters, providing insights into underlying patterns in the data.

## References

- https://scikit-learn.org/stable/modules/clustering.html#k-means
- Muller, A.C. & Guido, S. (2017) Introduction to Machine Learning with Python. A guide for Data scientists. USA: O'Reilly, chapter 3.
- VanderPlas, J. (2017) Python Data Science Handbook: Essential Tools for Working with Data. USA: O'Reilly Media, Inc. chapter 5.